<a href="https://colab.research.google.com/github/aaaksenova/NN_methods/blob/change/HW4_BertModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework4 BERT


## Data

In [1]:
!pip install -q -U watermark

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 2.1.3 which is incompatible.


In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.2 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
     |████████████████████████████████| 895 kB 42.0 MB/s 
     |████████████████████████████████| 596 kB 31.1 MB/s 
     |████████████████████████████████| 6.5 MB 35.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

Python implementation: CPython
Python version       : 3.7.12
IPython version      : 5.5.0

numpy       : 1.21.5
pandas      : 1.3.5
torch       : 1.10.0+cu111
transformers: 4.17.0



In [5]:
import transformers
from transformers import BertModel, AutoTokenizer, BertTokenizer, \
     PreTrainedTokenizerFast, AdamW, get_linear_schedule_with_warmup
import torch.nn.functional as F

import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import Dataset, DataLoader


In [7]:
! pip install datasets

     |████████████████████████████████| 325 kB 3.5 MB/s 
     |████████████████████████████████| 134 kB 36.0 MB/s 
     |████████████████████████████████| 212 kB 32.6 MB/s 
     |████████████████████████████████| 1.1 MB 40.3 MB/s 
     |████████████████████████████████| 127 kB 35.5 MB/s 
     |████████████████████████████████| 271 kB 45.7 MB/s 
     |████████████████████████████████| 94 kB 2.3 MB/s 
     |████████████████████████████████| 144 kB 32.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [8]:
from datasets import load_dataset

imdb_dataset = load_dataset('imdb')

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
imdb_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [10]:
imdb_dataset["train"]["text"][:10]

['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, e

In [11]:
set(imdb_dataset["train"]["label"])

{0, 1}

Cutting up the data

In [12]:
train_texts = imdb_dataset["train"]["text"][:2000] + imdb_dataset["train"]["text"][-2000:]
train_labels = imdb_dataset["train"]["label"][:2000] + imdb_dataset["train"]["label"][-2000:]
test_texts = imdb_dataset["test"]["text"][:1000] + imdb_dataset["test"]["text"][-1000:]
test_labels = imdb_dataset["test"]["label"][:1000] + imdb_dataset["test"]["label"][-1000:]

In [13]:
class_num = set(test_labels)

In [14]:
del imdb_dataset

Sampling the test set

In [15]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

Tokenization

In [16]:
tokenizer = BertTokenizer.from_pretrained('prajjwal1/bert-small')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [17]:
encoding = tokenizer.encode_plus(
  'My mom wanted to help Suzy',
  max_length=30,
  add_special_tokens=True, # Добавить '[CLS]' и '[SEP]'
  return_token_type_ids=True,
  padding='max_length',
  return_attention_mask=True,
  return_tensors='pt',  # Вернуть тензор PyTorch
  truncation=True
)
encoding

{'input_ids': tensor([[  101,  2026,  3566,  2359,  2000,  2393, 28722,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]])}

In [18]:
train_encodings = tokenizer(train_texts, truncation=True, 
                            padding=True, return_token_type_ids=False, 
                            return_attention_mask=True, max_length=512
                            )
val_encodings = tokenizer(val_texts, truncation=True, 
                          padding=True, return_token_type_ids=False,
                          return_attention_mask=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, 
                           padding=True, return_token_type_ids=False,
                           return_attention_mask=True, max_length=512)

Dataset processing

In [19]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

Metrics function

In [20]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

Training arguments for all the models

In [21]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

## BERT standard

In [22]:
from torch.nn import CrossEntropyLoss

In [23]:
class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):

    super().__init__()
    self.bert = BertModel.from_pretrained('prajjwal1/bert-small')
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
    self.n_classes = n_classes
  
  # copied from BertForSequenceClassification sourcecode
  def forward(self,        
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None
    ):

    outputs = self.bert(input_ids=input_ids,
                        attention_mask=attention_mask,
                        return_dict=False)
    pooled_output = outputs[1]
    output = self.drop(pooled_output)
    logits = self.out(output)

    loss = None
    if labels is not None:
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.n_classes), labels.view(-1))
    output = (logits,)
    return ((loss,) + output) if loss is not None else output

In [24]:
model = SentimentClassifier(2) 
model = model.to('cuda')

Downloading:   0%|          | 0.00/111M [00:00<?, ?B/s]

Some weights of the model checkpoint at prajjwal1/bert-small were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics = compute_metrics    # metrics to evaluate
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3200
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 800


Step,Training Loss
10,0.786700
20,0.746600
30,0.804600
40,0.713800
50,0.703700
60,0.743500
70,0.675800
80,0.674100
90,0.650800
100,0.668500


Saving model checkpoint to ./results/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=800, training_loss=0.44583703711628914, metrics={'train_runtime': 236.1196, 'train_samples_per_second': 27.105, 'train_steps_per_second': 3.388, 'total_flos': 0.0, 'train_loss': 0.44583703711628914, 'epoch': 2.0})

In [27]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 800
  Batch size = 16


{'epoch': 2.0,
 'eval_accuracy': 0.89875,
 'eval_f1': 0.903225806451613,
 'eval_loss': 0.31449568271636963,
 'eval_precision': 0.9021479713603818,
 'eval_recall': 0.9043062200956937,
 'eval_runtime': 9.5634,
 'eval_samples_per_second': 83.653,
 'eval_steps_per_second': 5.228}

Evaluation

In [28]:
trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")

***** Running Evaluation *****
  Num examples = 2000
  Batch size = 16


{'epoch': 2.0,
 'test_accuracy': 0.8565,
 'test_f1': 0.8555611474584801,
 'test_loss': 0.40848851203918457,
 'test_precision': 0.8611955420466059,
 'test_recall': 0.85,
 'test_runtime': 24.2835,
 'test_samples_per_second': 82.36,
 'test_steps_per_second': 5.148}

## BERT - CLS

In [35]:
class SentimentClassifier_cls(nn.Module):

  def __init__(self, n_classes):

    super().__init__()
    self.bert = BertModel.from_pretrained('prajjwal1/bert-small')
    self.drop = nn.Dropout(p=0.3)
    self.linear = nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size)
    self.out = nn.Linear(self.bert.config.hidden_size + 512, n_classes)
    self.n_classes = n_classes
  
  # copied from BertForSequenceClassification sourcecode
  def forward(self,        
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None
    ):

    last_hidden_state, pooled_output = self.bert(
                                  input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  return_dict=False)
    
    cls = last_hidden_state[:,0,:]
    pooled_output = self.linear(self.drop(pooled_output))
    stacked_layers = torch.hstack([cls, pooled_output])
    
    logits = self.out(stacked_layers)

    loss = None
    if labels is not None:
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.n_classes), labels.view(-1))
    output = (logits,)
    return ((loss,) + output) if loss is not None else output

In [36]:
model = SentimentClassifier_cls(2) 
model = model.to('cuda')

loading configuration file https://huggingface.co/prajjwal1/bert-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ac031779e2b4dd1d9da1e39c9d6a29fd45deea195eb3703a701d9c77f60abb4e.1257bb8f1f585038e86954d2560e36ca5c2dd98a8cde30fd22468940c911b672
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/prajjwal1/bert-small/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/facfdb1638fdec899406e0efd5c2c43ae4bbaf

In [37]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics = compute_metrics    # metrics to evaluate
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3200
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 800


Step,Training Loss
10,0.725900
20,0.725500
30,0.720600
40,0.709900
50,0.706300
60,0.725400
70,0.680400
80,0.658800
90,0.638500
100,0.688200


Saving model checkpoint to ./results/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=800, training_loss=0.43358559906482697, metrics={'train_runtime': 235.5934, 'train_samples_per_second': 27.165, 'train_steps_per_second': 3.396, 'total_flos': 0.0, 'train_loss': 0.43358559906482697, 'epoch': 2.0})

In [38]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 800
  Batch size = 16


{'epoch': 2.0,
 'eval_accuracy': 0.90375,
 'eval_f1': 0.9077844311377246,
 'eval_loss': 0.31929314136505127,
 'eval_precision': 0.9088729016786571,
 'eval_recall': 0.9066985645933014,
 'eval_runtime': 9.5329,
 'eval_samples_per_second': 83.92,
 'eval_steps_per_second': 5.245}

Evaluation

In [39]:
trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")

***** Running Evaluation *****
  Num examples = 2000
  Batch size = 16


{'epoch': 2.0,
 'test_accuracy': 0.867,
 'test_f1': 0.8672654690618762,
 'test_loss': 0.419700562953949,
 'test_precision': 0.8655378486055777,
 'test_recall': 0.869,
 'test_runtime': 23.7998,
 'test_samples_per_second': 84.034,
 'test_steps_per_second': 5.252}

## Model out of the box

In [40]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("prajjwal1/bert-small")
model = model.to('cuda')

loading configuration file https://huggingface.co/prajjwal1/bert-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ac031779e2b4dd1d9da1e39c9d6a29fd45deea195eb3703a701d9c77f60abb4e.1257bb8f1f585038e86954d2560e36ca5c2dd98a8cde30fd22468940c911b672
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/prajjwal1/bert-small/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/facfdb1638fdec899406e0efd5c2c43ae4bbaf

In [41]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics = compute_metrics    # metrics to evaluate
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3200
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 800


Step,Training Loss
10,0.708600
20,0.728800
30,0.740200
40,0.681200
50,0.671600
60,0.681000
70,0.662900
80,0.654000
90,0.659600
100,0.648600


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=800, training_loss=0.4288963495194912, metrics={'train_runtime': 232.0008, 'train_samples_per_second': 27.586, 'train_steps_per_second': 3.448, 'total_flos': 253117897113600.0, 'train_loss': 0.4288963495194912, 'epoch': 2.0})

In [42]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 800
  Batch size = 16


{'epoch': 2.0,
 'eval_accuracy': 0.89875,
 'eval_f1': 0.8998763906056861,
 'eval_loss': 0.3262692987918854,
 'eval_precision': 0.9309462915601023,
 'eval_recall': 0.8708133971291866,
 'eval_runtime': 9.5063,
 'eval_samples_per_second': 84.154,
 'eval_steps_per_second': 5.26}

Evaluation

In [43]:
trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")

***** Running Evaluation *****
  Num examples = 2000
  Batch size = 16


{'epoch': 2.0,
 'test_accuracy': 0.858,
 'test_f1': 0.8539094650205762,
 'test_loss': 0.42693597078323364,
 'test_precision': 0.8792372881355932,
 'test_recall': 0.83,
 'test_runtime': 23.7661,
 'test_samples_per_second': 84.153,
 'test_steps_per_second': 5.26}

## BERT CLS-pooled

In [59]:
class SentimentClassifier_cls_pooled(nn.Module):

  def __init__(self, n_classes):

    super().__init__()
    self.bert = BertModel.from_pretrained('prajjwal1/bert-small')
    self.drop = nn.Dropout(p=0.3)
    self.linear = nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size)
    self.out = nn.Linear(self.bert.config.hidden_size + 512, n_classes)
    self.n_classes = n_classes
  
  # copied from BertForSequenceClassification sourcecode
  def forward(self,        
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None
    ):

    last_hidden_state, pooled_output, hidden_states = self.bert(
                                                  input_ids=input_ids,
                                                  attention_mask=attention_mask,
                                                  output_hidden_states=True,
                                                  return_dict=False)
    all_hidden_states = torch.stack(hidden_states)
    cls_embeddings = torch.mean(all_hidden_states[:, :, 0], 0)
    pooled_output = self.linear(self.drop(pooled_output))
    stacked_layers = torch.hstack([cls_embeddings, pooled_output])
    
    logits = self.out(stacked_layers)

    loss = None
    if labels is not None:
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.n_classes), labels.view(-1))
    output = (logits,)
    return ((loss,) + output) if loss is not None else output

In [60]:
model = SentimentClassifier_cls_pooled(2) 
model = model.to('cuda')

loading configuration file https://huggingface.co/prajjwal1/bert-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ac031779e2b4dd1d9da1e39c9d6a29fd45deea195eb3703a701d9c77f60abb4e.1257bb8f1f585038e86954d2560e36ca5c2dd98a8cde30fd22468940c911b672
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/prajjwal1/bert-small/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/facfdb1638fdec899406e0efd5c2c43ae4bbaf

In [61]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics = compute_metrics    # metrics to evaluate
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3200
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 800


Step,Training Loss
10,0.788100
20,0.735900
30,0.776900
40,0.699000
50,0.661600
60,0.705000
70,0.663500
80,0.663300
90,0.655300
100,0.652700


Saving model checkpoint to ./results/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=800, training_loss=0.42908045336604117, metrics={'train_runtime': 234.447, 'train_samples_per_second': 27.298, 'train_steps_per_second': 3.412, 'total_flos': 0.0, 'train_loss': 0.42908045336604117, 'epoch': 2.0})

In [62]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 800
  Batch size = 16


{'epoch': 2.0,
 'eval_accuracy': 0.905,
 'eval_f1': 0.9073170731707317,
 'eval_loss': 0.33309945464134216,
 'eval_precision': 0.9253731343283582,
 'eval_recall': 0.8899521531100478,
 'eval_runtime': 9.5266,
 'eval_samples_per_second': 83.975,
 'eval_steps_per_second': 5.248}

Evaluation

In [63]:
trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")

***** Running Evaluation *****
  Num examples = 2000
  Batch size = 16


{'epoch': 2.0,
 'test_accuracy': 0.8595,
 'test_f1': 0.8550799381124291,
 'test_loss': 0.458339661359787,
 'test_precision': 0.8828541001064962,
 'test_recall': 0.829,
 'test_runtime': 23.9462,
 'test_samples_per_second': 83.521,
 'test_steps_per_second': 5.22}

## Quality estimation

Model | test_acc
---|---
BERT Standard| 0.861
BERT CLS | 0.865
**BERT CLS-pooled**| **0.882**
BERT Out of the box | 0.879

As we can see, the best configuration is BERT_CLS_pooled with mean pooling if CLS tokens over all hidden states.

Let's try out the last model for prediction

Pixel phone review

In [82]:
text = """holla guys! i was using this phone in 2019..
          So far from that time,everything is okay but it suddenly getting blanked and the 
          screen shows something like yah in a factory mode..
          and i try to click on factory result but it popped up a text "your device is unlocked" 
          and yah i just keeping it in my underwear shelf until now.....
          Who knows how to solve this freakin problem? 
          because i already sent it to some mobile technician but they also didnt know 
          how to fix it.So complicated! hope you guys can help me pleaseee!"""
encoding = tokenizer.encode_plus(
  text,
  max_length=512,
  add_special_tokens=True, 
  return_token_type_ids=True,
  padding='max_length',
  return_attention_mask=True,
  return_tensors='pt',  
  truncation=True
).to('cuda')

In [83]:
model(
  input_ids=encoding['input_ids'], 
  attention_mask=encoding['attention_mask'])[0].cpu().detach().numpy().argmax()

0

Spiderman movie review

In [80]:
text = """This movie Is everything I feared about the future of cinema. 
      This shallow, empty CGI infested crapshow. 
      The real actors screentime Is replaced by computerized Stars. 
      And I just can't get why? There's no realistic impact to any of the action scenes. 
      The so-called superheroes are made immortal and doesn't get hurt. 
      They stand up no matter what hit them, punch them. 
      There's no realism or consequence to anything that's going on. 
      Spiderman just like the last 2 movies are a visual display of horrific computerized 
      images that's all It ever will be and shows that earth Is popularized
      with nerd reviewers that can't the difference between good movies and bad movies.
        Who cares anymore If a movie looks more like videogames? 
      Nobody supposedly..0/10 worst muck I ever seen. 
      Not a single scene made me go -wow this was exciting."""
encoding = tokenizer.encode_plus(
  text,
  max_length=512,
  add_special_tokens=True, 
  return_token_type_ids=True,
  padding='max_length',
  return_attention_mask=True,
  return_tensors='pt',  
  truncation=True
).to('cuda')

In [81]:
model(
  input_ids=encoding['input_ids'], 
  attention_mask=encoding['attention_mask'])[0].cpu().detach().numpy().argmax()

0

Flo app review

In [86]:
text = """I've been using this app for years and haven't really found one I like more.
          I like the layout of it mostly. 
          I would really like is the option to store LH test strips. 
          It has the BBT option, it would be nice to also be able to put the strips in 
          to get an accurate ovulation day."""

encoding = tokenizer.encode_plus(
  text,
  max_length=512,
  add_special_tokens=True, 
  return_token_type_ids=True,
  padding='max_length',
  return_attention_mask=True,
  return_tensors='pt',  
  truncation=True
).to('cuda')

In [87]:
model(
  input_ids=encoding['input_ids'], 
  attention_mask=encoding['attention_mask'])[0].cpu().detach().numpy().argmax()

1